In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

np.random.seed(42)
plt.style.use("ggplot")
max_len = 50

In [ ]:
data= pd.read_csv("./ner_dataset.csv",encoding="latin1")

In [ ]:
# Fix the problem that Sentence number is missing a lot of times!
data = None # Magic here
data.head()

In [ ]:
print("Unique Words in corpus:",data['Word'].nunique())
print("Unique Tag in corpus:",data['Tag'].nunique())

In [ ]:
#Get a vocabulary of available words and tags. Add the word ENDPAD to the vocab to represent padding
words = list()
words.append("ENDPAD")
num_words = len(words)
tags = list()
num_tags = len(tags)

In [ ]:
class SentenceGetter(object):
  def __init__(self,data):
    self.n_sent = 1 #counter
    self.data = data
    agg_func = lambda s:[(w,p,t) for w,p,t in zip(s['Word'].tolist(),s['POS'].tolist(),s['Tag'].tolist())]
    self.grouped = self.data.groupby("Sentence #").apply(agg_func)
    self.sentences = [s for s in self.grouped]

getter = SentenceGetter(data)
sentences = getter.sentences   # This was each sentence is correctly formatted

In [ ]:
sentences[0]

In [ ]:
#Construct a word to index map and a tag to index map
word2idx =  {}
tag2idx  =  {}

In [ ]:
X = [[word2idx[w[0]] for w in s]for s in sentences]
X = pad_sequences(maxlen = max_len , sequences =X, padding='post', value =num_words-1)

y = [[tag2idx[w[2]] for w in s]for s in sentences]
y = pad_sequences(maxlen = max_len , sequences =y, padding='post', value =tag2idx["O"])
y = [to_categorical(i, num_classes=num_tags) for i in y]

# Split X and y into train and val sets.
x_train,x_test,y_train,y_test = None


In [ ]:
# Create the model, first an Embedding layer. We will add dropout to avoid overfitting, this is a good technique to reproduce in NLP.
input_word = Input(shape = (max_len,))
model = Embedding( )(input_word)  # Fill
model = SpatialDropout1D(0.1)(model)
# Add a Bi-LSTM layer. Specify that the previous layer had recurrent_dropout=0.1 such that it is aware.
model = None # Fill
out = TimeDistributed(Dense(num_tags,activation='softmax'))(model)  # This layer allows to save space by reusing the weights on every word
model = None
model.summary()

In [ ]:

# Compile and fit the model.
history = None

In [ ]:
# Evaluate the model to get the true accuracy
model.evaluate(x_test, np.array(y_test))

In [ ]:
# Perform a prediction on a sample sentence and evaluate if all tags were correct
i = np.random.randint(0, x_test.shape[0])
p = model.predict()  # Fill
# Fill